# Импорт библиотек

In [1]:
!pip install statsmodels yfinance

     |████████████████████████████████| 6.4 MB 8.0 MB/s 
     |████████████████████████████████| 63 kB 682 kB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [56]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
import xgboost as xgb

from tqdm import tqdm

from datetime import timedelta 

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.tsa.stattools import adfuller
from sklearn.ensemble import RandomForestRegressor

plt.rcParams["figure.figsize"] = (20,8)

import warnings
warnings.filterwarnings('ignore')

# Настройка автоматизированного пайплайна

In [4]:
# Определим функцию потерь, которую нужно минимизировать через настройку гиперпараметров моделей

def calc_loss(y, y_pred, key_rate):
  if y >= 0 and y_pred >= 0:
    if y >= y_pred:
      # Могли вложить в деривативы больше, пришлось профицит кидать под овернайт
      return (key_rate - 0.009) * (y - y_pred) - (key_rate + 0.005) * (y - y_pred)
    elif y < y_pred:
      # Получили доп доход с деривативов, но пришлось разницу занимать под процент
      return (key_rate + 0.005) * (y_pred - y) - (key_rate + 0.01) * (y_pred - y)
  elif y < 0 and y_pred < 0:
    if y >= y_pred:
      # Заработали на овернайте, но упустили возможность вложить в деривативы
      return (y_pred - y) * (key_rate + 0.005) + (y - y_pred) * (key_rate - 0.009)
    elif y < y_pred:
      # Пришлось занять под процент из-за ошибки прогноза
      return (y - y_pred) * (key_rate + 0.01)
  elif y >= 0 and y_pred <= 0:
    # Заняли излишек, с учетом излишка вложили под овернайт, но упустили возможность вложить в деривативы
    return (key_rate - 0.009) * (y - y_pred) - (key_rate + 0.005) * y
  elif y <= 0 and y_pred >= 0:
    # Вложили в деривативы по прогнозу, но ошиблись и пришлось много занимать
    return (key_rate + 0.005) * y_pred - (key_rate + 0.01) * (y_pred - y)

In [213]:
class Pipeline:
  
  def __init__(self, data, key_rate = None, best_model = None):

    # Инициализируем параметры
    self.data = data
    self.best_model = best_model

  def prepare_data(self):

    # Отбираем данные в будние дни, так как в выходные сальдо преимущественно нулевое
    self.data = pd.DataFrame(self.data['Balance'])
    self.data['Weekday'] = self.data.index.dayofweek + 1
    self.data = self.data[(self.data['Weekday'] != 6) & (self.data['Weekday'] != 7)]

    # Определяем даты начала и конца исходного датасета для загрузки других признаков
    start_date = self.data.index[0]
    end_date = self.data.index[-1]

    # Загружаем некоторые признаки из yfinance: курс доллара к рублю, курс нефти, индекс мосбиржи
    usd_rub = yf.download('RUB=X', start = start_date, end = end_date)['Adj Close']
    brent = yf.download('BZ=F', start = start_date, end = end_date)['Adj Close']
    moex = yf.download('IMOEX.ME', start = start_date, end = end_date)['Adj Close']

    # Парсим ключевую ставку с официального сайта ЦБ
    key_rate = pd.read_html('https://www.cbr.ru/hd_base/KeyRate/?UniDbQuery.Posted=True&UniDbQuery.From=09.01.2017&UniDbQuery.To=31.03.2021')[0]
    key_rate['Дата'] = pd.to_datetime(key_rate['Дата'], format = "%d.%m.%Y")
    key_rate['Key_Rate'] = key_rate['Ставка'] / 10000
    key_rate.sort_values('Дата', ascending = True, inplace = True)
    key_rate.index = key_rate['Дата']
    key_rate.drop(['Ставка'], axis = 1, inplace = True)
    key_rate.drop(['Дата'], axis = 1, inplace = True)

    # Объединяем все данные в один датафрейм, а пропуски заменяем через линейную интерполяцию
    self.data = pd.merge(left = self.data, right = usd_rub, how = 'left', left_on = self.data.index, right_on = usd_rub.index).set_index('key_0').rename(columns = {'Adj Close': 'USD_RUB'})
    self.data = pd.merge(left = self.data, right = brent, how = 'left', left_on = self.data.index, right_on = brent.index).set_index('key_0').rename(columns = {'Adj Close': 'Brent'})
    self.data = pd.merge(left = self.data, right = moex, how = 'left', left_on = self.data.index, right_on = moex.index).set_index('key_0').rename(columns = {'Adj Close': 'MOEX'})
    self.data = pd.merge(left = self.data, right = key_rate, how = 'left', left_on = self.data.index, right_on = key_rate.index).set_index('key_0')
    self.data.interpolate(method = 'linear', inplace = True)
  
    # Возвращаем датафрейм со всеми загруженными признаками
    return self.data

  def generate_features(self):

    # Лаги зависимой переменной
    self.data['Balance_lag_1'] = self.data['Balance'].shift(1)
    self.data['Balance_lag_2'] = self.data['Balance'].shift(2)
    self.data['Balance_lag_3'] = self.data['Balance'].shift(3)
    self.data['Balance_lag_4'] = self.data['Balance'].shift(4)
    self.data['Balance_lag_5'] = self.data['Balance'].shift(5)

    # Скользящие средние зависимой переменной
    self.data['Balance_Rolling_Mean_5'] = self.data['Balance'].rolling(window = 5).mean()
    self.data['Balance_Rolling_Mean_15'] = self.data['Balance'].rolling(window = 15).mean()
    self.data['Balance_Rolling_Mean_30'] = self.data['Balance'].rolling(window = 30).mean()
    self.data['Balance_Rolling_Mean_45'] = self.data['Balance'].rolling(window = 45).mean()
    self.data['Balance_Rolling_Mean_60'] = self.data['Balance'].rolling(window = 60).mean()

    # Фиктивные переменные дня недели
    self.data = pd.get_dummies(self.data, columns = ['Weekday'], drop_first = True)
    self.data.dropna(inplace = True)

    # Возвращаем датафрейм с загруженными признаками и сгенерированными из имеющихся рядов фичами
    return self.data

  def select_features(self):

    # Разбиваем данные для использования библиотеки по отбору признаков
    key_rate = self.data['Key_Rate'] 
    X = self.data.drop(['Balance'], axis = 1)
    y = self.data['Balance']

    # Обучаем модель для отбора лучших 5 признаков
    X_kbest = SelectKBest(f_regression, k=5).fit_transform(X, y)
    data_kbest_1 = pd.DataFrame(X_kbest, index = X.index)
    data_kbest_1['Balance'] = y
    data_kbest_1['Key_Rate'] = key_rate
    self.data_kbest = data_kbest_1

    # Обучаем PCA модель, сохраняя 90% вариации параметров
    scaler = StandardScaler()
    scaler.fit(X)
    X_scaler = scaler.transform(X)
    pca = PCA(.9)
    pca.fit(X_scaler)
    X_pca = pca.transform(X_scaler)
    data_pca_1 = pd.DataFrame(X_pca, index = X.index)
    data_pca_1['Balance'] = y
    data_pca_1['Key_Rate'] = key_rate
    self.data_pca = data_pca_1

    # Возвращаем словарь с двумя датафреймами с выделенными параметрами
    return {'data_kbest': self.data_kbest, 'data_pca': self.data_pca}

  def best_arima_model(self, data):

    # Разбиваем исходный датафрейм
    key_rate = data['Key_Rate']
    X = data.drop(['Balance', 'Key_Rate'], axis = 1)
    y = data['Balance']

    # Ищем оптимальные параметры ARIMA
    self.best_d = 0
    y_temp = y
    while adfuller(y_temp)[1] > 0.05:
      self.best_d += 1
      y_temp = y_temp.diff().dropna()
    
    self.best_p = 0
    self.best_q = 0
    best_errors_list = []
    best_loss = 9999
    start_index = 0
    end_index = 500
    step = 10
    for p in tqdm(range(0, 1)):
      for q in range(0, 1):

        steps = list(range(end_index, len(X) - 1, 10))

        errors_list = []
        for i, step in enumerate(steps):
          X_train = X.iloc[i*10 : step, :]
          X_test = X.iloc[step : step+1, :]

          y_train = y.iloc[i*10 : step]
          y_test = y.iloc[step : step+1]

          model = SARIMAX(y_train, X_train, order = (p, self.best_d, q))
          result = model.fit()
          y_pred = result.get_forecast(steps = 1, exog = np.array(X_test.iloc[-1]).reshape((1,-1))).predicted_mean

          error = calc_loss(float(y_test), float(y_pred), key_rate.loc[y_test.index])
          errors_list.append(abs(error))
        
        loss = np.mean(errors_list)
        if loss < best_loss:
          best_loss = loss
          best_errors_list = errors_list
          self.best_p = p
          self.best_q = q

    # Возвращаем оптимальную для ARIMA среднюю ошибку, максимальную ошибку в модели и массив гиперпараметров
    return [best_loss, np.max(best_errors_list), 'ARIMA', [self.best_p, self.best_q]]

  def best_xgb_model(self, data):

    # Первичное деление готового датафрейма
    key_rate = data['Key_Rate']
    X = data.drop(['Balance', 'Key_Rate'], axis = 1)
    y = data['Balance']

    # Определяем пространство возможных гиперпараметров
    lr_list = np.linspace(1e-4, 1e-1, 2)
    max_depth_list = np.array([3])
    n_estimators_list = np.array([300])

    # Ищем оптимальные параметры XGBoost
    self.best_lr = 0
    self.best_max_depth = 0
    self.best_n_estimators = 0
    best_errors_list = []
    best_loss = 9999
    start_index = 0
    end_index = 500
    step = 10
    for lr in tqdm(lr_list):
      for max_depth in max_depth_list:
        for n_estimators in n_estimators_list:

          steps = list(range(end_index, len(X) - 1, 10))

          errors_list = []
          for i, step in enumerate(steps):
            X_train = X.iloc[i*10 : step, :]
            X_test = X.iloc[step : step+1, :]

            y_train = y.iloc[i*10 : step]
            y_test = y.iloc[step : step+1]

            model = xgb.XGBRegressor(learning_rate = lr, max_depth = max_depth, n_estimators = n_estimators, verbosity = 0)
            result = model.fit(X_train, y_train)
            y_pred = result.predict(X_test)

            error = calc_loss(y_test[0], y_pred[0], key_rate.loc[y_test.index].values)
            errors_list.append(abs(error))
        
        loss = np.mean(errors_list)
        if loss < best_loss:
          best_loss = loss
          best_errors_list = errors_list
          self.best_lr = lr
          self.best_max_depth = max_depth
          self.best_n_estimators = n_estimators

    # Возвращаем оптимальную для XGboost среднюю ошибку, максимальную ошибку в модели и массив гиперпараметров
    return [best_loss, np.max(best_errors_list), 'XGBoost', [self.best_lr, self.best_max_depth, self.best_n_estimators]]

  def best_rf_model(self, data):

    # Первичное деление готового датафрейма
    key_rate = data['Key_Rate']
    X = data.drop(['Balance', 'Key_Rate'], axis = 1)
    y = data['Balance']

    # Определяем пространство возможных гиперпараметров
    min_samples_split_list = np.array([4])
    min_samples_leaf_list = np.array([5])
    max_depth_list = np.array([4])
    n_estimators_list = np.array([300])

    # Ищем оптимальные параметры RandomForest
    self.best_min_samples_split = 0
    self.best_min_samples_leaf = 0
    self.best_max_depth = 0
    self.best_n_estimators = 0
    best_errors_list = []
    best_loss = 9999
    start_index = 0
    end_index = 500
    step = 10
    for min_samples_leaf in tqdm(min_samples_leaf_list):
      for min_samples_split in min_samples_split_list:
        for max_depth in max_depth_list:
          for n_estimators in n_estimators_list:

            steps = list(range(end_index, len(X) - 1, 10))

            errors_list = []
            for i, step in enumerate(steps):
              X_train = X.iloc[i*10 : step, :]
              X_test = X.iloc[step : step+1, :]

              y_train = y.iloc[i*10 : step]
              y_test = y.iloc[step : step+1]

              model = RandomForestRegressor(min_samples_leaf = min_samples_leaf,
                                            min_samples_split = min_samples_split,
                                            max_depth = max_depth, 
                                            n_estimators = n_estimators, 
                                            verbose = 0)
              
              result = model.fit(X_train, y_train)
              y_pred = result.predict(X_test)

              error = calc_loss(y_test[0], y_pred[0], key_rate.loc[y_test.index].values)
              errors_list.append(abs(error))
          
            loss = np.mean(errors_list)
            if loss < best_loss:
              best_loss = loss
              best_errors_list = errors_list
              self.best_min_samples_leaf = min_samples_leaf
              self.best_min_samples_split = min_samples_split
              self.best_max_depth = max_depth
              self.best_n_estimators = n_estimators
    
    # Возвращаем оптимальную для RandomForest среднюю ошибку, максимальную ошибку в модели и массив гиперпараметров
    return [best_loss, np.max(best_errors_list), 'RandomForest', [self.best_min_samples_leaf, self.best_min_samples_split, self.best_max_depth, self.best_n_estimators]]

  def find_best_model(self, data, start_index = 0, end_index = 500, step = 10):

    # Запускаем поиск оптимальных гиперпараметров ARIMA, XGB и RF моделей
    best_arima = self.best_arima_model(data)
    best_xgb = self.best_xgb_model(data)
    best_rf = self.best_rf_model(data)

    best_models = [best_arima, best_xgb, best_rf]

    # Определяем лучшую модель из трех лучших
    best_score = 9999
    best_index = 0
    for i, model in enumerate(best_models):
      model_score = model[0]
      if model_score > best_score:
        best_score = model_score
        best_index = i
    
    # Возвращаем характеристики лучшей модели
    return best_models[best_index]

  def activate_pipeline(self, start_index = 0, end_index = 500, step = 10):
    
    # Определяем интервал для обучения при backtesting
    self.start_index = start_index
    self.end_index = end_index
    self.step = step

    # Проводим первичные операции над исходным датафреймом
    self.data = self.prepare_data()
    self.data = self.generate_features()
    
    # Производим отбор признаков через два метода, определяем лучшие модели и лучший вариант отбора
    dict_df = self.select_features()
    best_loss = 9999
    best_model = None
    for df_selected in dict_df.keys():
      self.best_model = self.find_best_model(dict_df[df_selected])
      if self.best_model[0] < best_loss:
        self.best_selected_df = dict_df[df_selected]
        best_loss = self.best_model[0]
        best_model = self.best_model

    # Возвращаем характеристики лучшей модели и датафрейм с отобранными эффективным способом признаками
    return [self.best_model, self.best_selected_df]
  
  def predict_1_day(self, prediction_date = None):
    
    # Если не указана дата желаемого прогноза
    if prediction_date is None:

      # Если дата, следующая за последним индексом датафрейма попадает на выходной день
      if ((self.best_selected_df.index[-1] + timedelta(days=1)).dayofweek + 1 == 6) or \
         ((self.best_selected_df.index[-1] + timedelta(days=1)).dayofweek + 1 == 7):

        return 0
    
      else:
        data = self.best_selected_df
        key_rate = data['Key_Rate']
        X = data.drop(['Balance', 'Key_Rate'], axis = 1)
        y = data['Balance']

        X.loc[X.index[-1] + timedelta(days = 1)] = [np.nan] * X.shape[1]
        X.interpolate(method = 'linear', inplace = True)

        X_train = X.iloc[:-1, :]
        X_test = X.iloc[-1:, :]

        y_train = y

        # Ситуация, когда в лучшей модели 2 гиперпараметра - это ARIMA модель
        if len(self.best_model[3]) == 2:
          model = SARIMAX(y_train, X_train, order = (self.best_p, self.best_d, self.best_q))
          result = model.fit()
          y_pred = result.get_forecast(steps = 1, exog = np.array(X_test.iloc[-1]).reshape((1,-1))).predicted_mean
          if y_pred[0] < y_train.quantile(0.95) and y_pred[0] > y_train.quantile(0.05):
            return y_pred[0]
          else:
            return f'Прогноз {y_pred[0]} является неустойчивым относительно выборки, требуется ручная настройка и анализ потоков ликвидности'

        # Ситуация, когда в лучшей модели 3 гиперпараметра - это XGBoost модель
        elif len(self.best_model[3]) == 3:
          model = xgb.XGBRegressor(learning_rate = self.best_lr, max_depth = self.best_max_depth, n_estimators = self.best_n_estimators)
          result = model.fit(X_train, y_train)
          y_pred = result.predict(X_test)
          if y_pred[0] < y_train.quantile(0.95) and y_pred[0] > y_train.quantile(0.05):
            return y_pred[0]
          else:
            return f'Прогноз {y_pred[0]} является неустойчивым относительно выборки, требуется ручная настройка и анализ потоков ликвидности'

        # Ситуация, когда в лучшей модели 4 гиперпараметра - это RandomForest модель
        elif len(self.best_model[3]) == 4:
          model = RandomForestRegressor(min_samples_leaf = self.best_min_samples_leaf,
                                              min_samples_split = self.best_min_samples_split,
                                              max_depth = self.best_max_depth, 
                                              n_estimators = self.best_n_estimators, 
                                              verbose = 0)
                
          result = model.fit(X_train, y_train)
          y_pred = result.predict(X_test)
          if y_pred[0] < y_train.quantile(0.95) and y_pred[0] > y_train.quantile(0.05):
            return y_pred[0]
          else:
            return f'Прогноз {y_pred[0]} является неустойчивым относительно выборки, требуется ручная настройка и анализ потоков ликвидности'

    else:

      # Настраиваем дату в нужный формат. Предварительно даты заносятся в виде 'dd.mm.YYYY'
      prediction_date = pd.to_datetime(prediction_date, format = '%d.%m.%Y')

      # Если указанная дата попала на выходной день
      if (prediction_date.dayofweek + 1 == 6) or (prediction_date.dayofweek + 1 == 7):
        return 0
      
      # Если указанная дата представляет собой день, следующий за последним индексом датафрейма
      elif prediction_date == self.best_selected_df.index[-1] + timedelta(days=1):    
        data = self.best_selected_df
        key_rate = data['Key_Rate']
        X = data.drop(['Balance', 'Key_Rate'], axis = 1)
        y = data['Balance']

        X.loc[X.index[-1] + timedelta(days = 1)] = [np.nan] * X.shape[1]
        X.interpolate(method = 'linear', inplace = True)

        X_train = X.iloc[:-1, :]
        X_test = X.iloc[-1:, :]

        y_train = y

        # Ситуация, когда в лучшей модели 2 гиперпараметра - это ARIMA модель
        if len(self.best_model[3]) == 2:
          model = SARIMAX(y_train, X_train, order = (self.best_p, self.best_d, self.best_q))
          result = model.fit()
          y_pred = result.get_forecast(steps = 1, exog = np.array(X_test.iloc[-1]).reshape((1,-1))).predicted_mean
          if y_pred[0] < y_train.quantile(0.95) and y_pred[0] > y_train.quantile(0.05):
            return y_pred[0]
          else:
            return f'Прогноз {y_pred[0]} является неустойчивым относительно выборки, требуется ручная настройка и анализ потоков ликвидности'

        # Ситуация, когда в лучшей модели 3 гиперпараметра - это XGBoost модель
        elif len(self.best_model[3]) == 3:
          model = xgb.XGBRegressor(learning_rate = self.best_lr, max_depth = self.best_max_depth, n_estimators = self.best_n_estimators)
          result = model.fit(X_train, y_train)
          y_pred = result.predict(X_test)
          if y_pred[0] < y_train.quantile(0.95) and y_pred[0] > y_train.quantile(0.05):
            return y_pred[0]
          else:
            return f'Прогноз {y_pred[0]} является неустойчивым относительно выборки, требуется ручная настройка и анализ потоков ликвидности'

        # Ситуация, когда в лучшей модели 4 гиперпараметра - это RandomForest модель
        elif len(self.best_model[3]) == 4:
          model = RandomForestRegressor(min_samples_leaf = self.best_min_samples_leaf,
                                              min_samples_split = self.best_min_samples_split,
                                              max_depth = self.best_max_depth, 
                                              n_estimators = self.best_n_estimators, 
                                              verbose = 0)
                
          result = model.fit(X_train, y_train)
          y_pred = result.predict(X_test)
          if y_pred[0] < y_train.quantile(0.95) and y_pred[0] > y_train.quantile(0.05):
            return y_pred[0]
          else:
            return f'Прогноз {y_pred[0]} является неустойчивым относительно выборки, требуется ручная настройка и анализ потоков ликвидности'

      else:
        data = self.best_selected_df
        key_rate = data['Key_Rate']
        X = data.drop(['Balance', 'Key_Rate'], axis = 1)
        y = data['Balance']

        # Когда дата прогноза есть в индексе в имеющимся датафрейме
        if prediction_date in X.index:
          
          # Модель обучается до момента даты прогноза
          X_train = X.loc[:(prediction_date + timedelta(days=1)), :]
          X_test = X.loc[prediction_date:prediction_date, :]

          # Модель прогнозируется на следующее наблюдение
          y_train = y.loc[:(prediction_date + timedelta(days=1))]
          y_test = y.loc[prediction_date:prediction_date]

          # Ситуация, когда в лучшей модели 2 гиперпараметра - это ARIMA модель
          if len(self.best_model[3]) == 2:
            model = SARIMAX(y_train, X_train, order = (self.best_p, self.best_d, self.best_q))
            result = model.fit()
            y_pred = result.get_forecast(steps = 1, exog = np.array(X_test.iloc[-1]).reshape((1,-1))).predicted_mean
            if y_pred[0] < y_train.quantile(0.95) and y_pred[0] > y_train.quantile(0.05):
              return y_pred[0]
            else:
              return f'Прогноз {y_pred[0]} является неустойчивым относительно выборки, требуется ручная настройка и анализ потоков ликвидности'

          # Ситуация, когда в лучшей модели 3 гиперпараметра - это XGBoost модель
          elif len(self.best_model[3]) == 3:
            model = xgb.XGBRegressor(learning_rate = self.best_lr, max_depth = self.best_max_depth, n_estimators = self.best_n_estimators)
            result = model.fit(X_train, y_train)
            y_pred = result.predict(X_test)
            if y_pred[0] < y_train.quantile(0.95) and y_pred[0] > y_train.quantile(0.05):
              return y_pred[0]
            else:
              return f'Прогноз {y_pred[0]} является неустойчивым относительно выборки, требуется ручная настройка и анализ потоков ликвидности'

          # Ситуация, когда в лучшей модели 4 гиперпараметра - это RandomForest модель
          elif len(self.best_model[3]) == 4:
            model = RandomForestRegressor(min_samples_leaf = self.best_min_samples_leaf,
                                                min_samples_split = self.best_min_samples_split,
                                                max_depth = self.best_max_depth, 
                                                n_estimators = self.best_n_estimators, 
                                                verbose = 0)
                  
            result = model.fit(X_train, y_train)
            y_pred = result.predict(X_test)
            if y_pred[0] < y_train.quantile(0.95) and y_pred[0] > y_train.quantile(0.05):
              return y_pred[0]
            else:
              return f'Прогноз {y_pred[0]} является неустойчивым относительно выборки, требуется ручная настройка и анализ потоков ликвидности'

        else:
          return 'Введенная дата недоступна для прогноза'

# Проверка работоспособности алгоритма

In [143]:
# Создаем экземпляр класса с нашим исходным датафреймом
example = Pipeline(pd.read_excel('/content/Project 2_2022.xlsx', parse_dates = ['Date'], index_col = 'Date'))

In [144]:
# Активируем пайплайн
result = example.activate_pipeline()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


100%|██████████| 3/3 [36:34<00:00, 731.51s/it]


In [145]:
# Смотрим на результат, где первый элемент массива - параметры лучшей модели (средняя абсолютная ошибка, 
# максимальная абсолютная ошибка, название модели и массив гиперпарамтеров), а второй - датафрейм с отобранными признаками

print('Средняя абсолютная ошибка:', result[0][0])
print('Максимальная абсолютная ошибка:', result[0][1])
print('Название модели:', result[0][2])

if result[0][2] == 'ARIMA':
  print('Параметр p ARIMA модели:', result[0][3][0])
  print('Параметр q ARIMA модели:', result[0][3][1])
  
elif result[0][2] == 'XGBoost':
  print('Параметр learning_rate XGBoost модели:', result[0][3][0])
  print('Параметр max_depth XGBoost модели:', result[0][3][1])
  print('Параметр n_estimators XGBoost модели:', result[0][3][2])

elif result[0][2] == 'RandomForest':
  print('Параметр min_samples_leaf RandomForest модели:', result[0][3][0])
  print('Параметр min_samples_split RandomForest модели:', result[0][3][1])
  print('Параметр max_depth RandomForest модели:', result[0][3][2])
  print('Параметр n_estimators RandomForest модели:', result[0][3][3])

Средняя абсолютная ошибка: 0.00834836577989709
Максимальная абсолютная ошибка: 0.09540074493647305
Название модели: ARIMA
Параметр p ARIMA модели: 0
Параметр q ARIMA модели: 3


* Как видим, лучше всего себя во время backtesting проявила модель MA(3). Для нее средняя абсолютная ошибка нашей метрики равняется 0.008, а максимальная ошибка - около 0.1, что удовлетворяет потребностям бизнеса

In [215]:
# Пробуем разные прогнозы
print('Прогноз на 1 день вперед:', example.predict_1_day())
print('Прогноз на произвольный будний день внутри выборки:', example.predict_1_day('22.04.2020'))
print('Прогноз на произвольный будний день вне выборки:', example.predict_1_day('24.02.2022'))
print('Прогноз на произвольный выходной день внутри выборки:', example.predict_1_day('25.02.2020'))

Прогноз на 1 день вперед: Прогноз -0.9136592714463844 является неустойчивым относительно выборки, требуется ручная настройка и анализ потоков ликвидности
Прогноз на произвольный будний день внутри выборки: -0.12105507380352797
Прогноз на произвольный будний день вне выборки: Введенная дата недоступна для прогноза
Прогноз на произвольный выходной день внутри выборки: -0.08112575553237784
